In [1]:
# import required libraries
import pandas as pd
import numpy as np
from openpyxl import Workbook
import json
import re
import os

In [2]:
# set the relative path for the exports
# -------------------------------------
# base path choice on OS
print(f"Operating System detected: {os}")

if os.name == 'nt':  # Windows
    rel_path = "C://Users/ben_g/OneDrive - King County Regional Homelessness Authority/Research and Data - 2026 Unsheltered PIT/Volunteer Training & Schedule/Doodle Exports"
    # set path for output
    filepath = 'C://Users/ben_g/OneDrive - King County Regional Homelessness Authority/Research and Data - 2026 Unsheltered PIT/Volunteer Training & Schedule/Volunteer Schedules - Doodle.xlsx'
else:  # macOS or Linux
    rel_path = "/Users/ben.mathewson/Library/CloudStorage/OneDrive-KingCountyRegionalHomelessnessAuthority/Research and Data - 2026 Unsheltered PIT/Volunteer Training & Schedule/Doodle Exports"
    # set path for output
    filepath = '/Users/ben.mathewson/Library/CloudStorage/OneDrive-KingCountyRegionalHomelessnessAuthority/Research and Data - 2026 Unsheltered PIT/Volunteer Training & Schedule/Volunteer Schedules - Doodle.xlsx'

Operating System detected: <module 'os' (frozen)>


In [3]:
display(filepath)
display(rel_path)

'/Users/ben.mathewson/Library/CloudStorage/OneDrive-KingCountyRegionalHomelessnessAuthority/Research and Data - 2026 Unsheltered PIT/Volunteer Training & Schedule/Volunteer Schedules - Doodle.xlsx'

'/Users/ben.mathewson/Library/CloudStorage/OneDrive-KingCountyRegionalHomelessnessAuthority/Research and Data - 2026 Unsheltered PIT/Volunteer Training & Schedule/Doodle Exports'

In [4]:
%%capture
# loop over the list of workbooks
subregions = [
    "Vashon Island",
    "South Seattle",
    "South King County",
    "South East King County",
    "Snoqualmie Valley",
    "North Seattle",
    "North King County",
    "Family Phone Line",
    "East King County",
    "Downtown Seattle"
]

# ensure the working file exists
if not os.path.exists(filepath):
    # If neither exists, create a fresh empty workbook
    wb = Workbook()
    wb.save(filepath)
    print("Created new working file.")

# create dataframe to concatentate workbooks into
all_regions = pd.DataFrame()

# loop over the list, creating an Excel workbook of results
# --------------
for region in subregions:
    # get the source export
    path = f"{rel_path}/{region}.xlsx" # MAY NEED TO SPECIFY
    # read into a pandas dataframe
    workbook = pd.read_excel(io=path,
                            header=5,
                            index_col=0,
                            engine='openpyxl')
    
    # do a little cleaning
    workbook_stripped = workbook.drop(columns=["email",
                                    "Volunteer Emergency Contact Name (Last, First)",
                                    "Volunteer Emergency Contact Phone Number",
                                    "Volunteer Phone Number",
                                    "Waiver Completed?"])
    
    # transpose the dataframe
    workbook_stripped = workbook_stripped.T

    # reset the index
    workbook_stripped = workbook_stripped.reset_index(names="DateTime")

    # split 'DateTime' into two columns
    # ----------------------------------
    # look for a space preceded by 4 digits
    split_data = workbook_stripped['DateTime'].str.split(r'(?<=\d{4})\s', expand=True)

    workbook_stripped['Date'] = split_data[0]
    workbook_stripped['Time'] = split_data[1]

    # groupby columns to handle duplicate columns
    workbook_stripped = workbook_stripped.groupby(level=0,
                                                axis=1).first()
    
    # replace all "Signed-up" with "Time"
    for col in workbook_stripped.columns:
        # print(col)
        workbook_stripped.loc[workbook_stripped[col] == "Signed-up", col] = workbook_stripped['Time']

    # drop newly extraneous columns
    workbook_stripped = workbook_stripped.drop(columns=["DateTime",
                                                        "Time",
                                                        "Total seats",
                                                        "Participants signed-up"])

    # reset the index to be the "Date" col (ahead of transposing)
    workbook_stripped = workbook_stripped.set_index(keys="Date")

    # add the subregion name as a feature
    workbook_stripped["Subregion"] = region

    # concatentate to the big dataframe
    all_regions = pd.concat([all_regions, workbook_stripped])

    # loop over the locations to create dataframes similar to the excel tracker
    # ------------------
    # get a list of unique locations
    locations = list(set(workbook_stripped["Session description"]))

    # create a dictionary to hold the indexed dataframes
    dfs = {}

    # loop over each location to create a separate, named df
    for loc in locations:
        # create dataframes as transposed copies
        # -------------------
        # filter for target location
        df = workbook_stripped[workbook_stripped["Session description"]==loc].copy()

        # drop the "Session description" from the newly created df
        df = df.drop(columns=["Session description"])

        # convert the "Date" column to datetime
        df.index = pd.to_datetime(df.index,
                                    errors="coerce")

        # transpose the dataframe and drop empty rows
        df = df.T.dropna(how='all')

        # group by column and take the 'first' non-null entries
        df = df.groupby(level=0,
                        axis=1).first()
        
        # order the dataframe columns chronologically
        df = df.sort_index(axis=1)

        # reset the index so we don't lose names
        df = df.reset_index()

        # drop any duplicate rows
        df = df.drop_duplicates()

        # remove the row containing 'Subregion'
        df = df.drop(index=df[df["Name"]=="Subregion"].index, axis=0)

        # join back to 'workbook' to preserve email and phone number
        workbook = workbook.reset_index() # reset index to restore 'Name' as a column
        workbook = workbook[['Name',
                            'Volunteer Phone Number',
                            'email']] # keep only essential information
        df = pd.merge(left=df,
                    right=workbook,
                    on="Name",
                    how="left") # merge the dataframes to capture contact info
        
        # add placeholder column 'Confirmed'
        df["Confirmed"] = "" # no content, will appear empty
        
        # build sorting function for columns
        def column_sort_key(col):
            if isinstance(col, str):
                # (Group 0, Column Name)
                return (0, col)
            else:
                # (Group 1, Timestamp Value)
                return (1, col)

        # sort name and contact info to the front
        new_order = sorted(df.columns, key=column_sort_key)
        df = df[new_order]

        # change and standardize column labels (format)
        df = df.rename(mapper={"Volunteer Phone Number":"Phone",
                            "email":"Email"},
                    axis=1)
 
        # limit the sheet name len to avoid errors
        if len(loc)>31:
            loc = str(loc[:31])

        # Format Timestamp columns to 'Short Date' + (Day Abbreviation)
        # Result: 2026-01-26 becomes "1/26/26 (Mon)"
        new_column_names = []
        for col in df.columns:
            if isinstance(col, pd.Timestamp):
                # %m/%d/%y -> 01/26/26
                # %a -> Mon
                formatted_date = col.strftime('%m/%d/%y (%a)')
                new_column_names.append(formatted_date)
            else:
                new_column_names.append(col)
        
        df.columns = new_column_names

        # WRITE TO EXCEL WORKBOOK
        # -------------------
        # create the ExcelWriter object
        with pd.ExcelWriter(filepath,
                            engine='openpyxl',
                            mode='a',
                            if_sheet_exists='replace') as writer:
            # index=False prevents the 0, 1, 2... row numbers from being saved
            df.to_excel(writer, sheet_name=loc, index=False)

            # access the openpyxl worksheet object for this specific sheet
            worksheet = writer.sheets[loc]
            
            # iterate through all columns and set the width
            # Excel columns are 1-indexed (A=1, B=2, etc.)
            for i, col in enumerate(df.columns, 1):
                column_letter = worksheet.cell(row=1, column=i).column_letter
                
                # set width to 25 (Excel units)
                worksheet.column_dimensions[column_letter].width = 25

# Transform Compiled Workbooks for Dashboard

In [5]:
# reset the index so that we can manipulate 'Date'
all_regions = all_regions.reset_index(names="Date")

# convert 'Date' column to datetime obj
all_regions["Date"] = pd.to_datetime(all_regions["Date"], errors='coerce').dt.strftime('%m/%d/%Y')

In [6]:
# create a list of volunteer names (for cross-ref to smartsheet)
volunteers = list(set(all_regions.columns))

In [7]:
# set 'Date' and 'Session description' as multiindex before grouping columns
all_regions = all_regions.set_index(keys=["Date", "Session description", "Subregion"])

In [8]:
# groupby columns to handle duplicate columns
grouped_wbk = all_regions.stack().groupby(level=[0,1,2]).value_counts().reset_index(name="Count")

In [9]:
# create function to handle the time transformation
def parse_shift_times(df, column_name='level_3'):
    # clean the string: remove the ".1" duplicates and split by newline
    # This separates "10:00 AM" from "4 h"
    temp_split = df[column_name].str.replace(r'\.\d+$', '', regex=True).str.split('\n', expand=True)
    
    # convert the first part to a datetime (Shift Start)
    df['Shift Start'] = pd.to_datetime(temp_split[0], format='%I:%M %p').dt.time
    
    # convert the second part to a Timedelta (Duration)
    # clean the duration string to make it compatible with pandas (e.g., "4 h 30 min" -> "4h 30m")
    duration_str = temp_split[1].str.replace(' h', 'h').str.replace(' min', 'm')
    duration_td = pd.to_timedelta(duration_str)
    
    # calculate Shift End
    # combine a dummy date with the start time to do the math, then extract the time
    dummy_date = pd.to_datetime('2026-01-01 ' + temp_split[0])
    df['Shift End'] = (dummy_date + duration_td).dt.time

    # drop the source column
    df = df.drop(columns=["level_3"])
    
    return df

# apply the function to your 'grouped_wbk' dataframe
grouped_wbk = parse_shift_times(grouped_wbk)

# Create JSON Representation for Export

In [10]:
# define the color palette by site
# (use limited palette, sites by subregion)
site_colors = {
    # Downtown Seattle
    "Mary's Place":"#172B69",
    "St. James Cathedral":"#1B8477",
    "Youthcare Orion":"#A5C04D",
    "Compass Day Center":"#FFD600",
    # East King County
    "Bellevue Library":"#172B69",
    "Issaquah Community Hall":"#1B8477",
    "Overlake Christian Church":"#A5C04D",
    "Kirkland Library":"#FFD600",
    # Family Phone Line
    "Mary's Place Phone Line":"#172B69",
    # North King County
    "Shoreline Library":"#172B69",
    "Ronald United Methodist Church":"#1B8477",
    # North Seattle
    "North Seattle College":"#172B69",
    "Lake City Library":"#1B8477",
    # Snoqualmie Valley
    "Reclaim":"#172B69",
    "North Bend Library":"#1B8477",
    # South East King County
    "Maple Valley Food Bank":"#172B69",
    "Plateau Outreach Ministries":"#1B8477",
    # South King County
    "Kent Library":"#172B69",
    "Highline United Methodist Church":"#1B8477",
    "Federal Way Library":"#A5C04D",
    "Renton Library":"#FFD600",
    # South Seattle
    "Southwest Library":"#172B69",
    "Georgetown SVP":"#1B8477",
    "South Lucille VA Center":"#A5C04D",
    "South Park Library":"#FFD600",
    # Vashon Island
    "Vashon Island Library":"#172B69",
    "Vashon Food Bank":"#1B8477"
}

In [11]:
# create ISO 8601 strings for Start and End
# assumes 'Date' is already a string or datetime object
grouped_wbk['iso_start'] = pd.to_datetime(grouped_wbk['Date'] \
                                          .astype(str) + \
                                            ' ' + \
                                                grouped_wbk['Shift Start'] \
                                                    .astype(str)).dt \
                                                        .strftime('%Y-%m-%dT%H:%M:%S')
grouped_wbk['iso_end'] = pd.to_datetime(grouped_wbk['Date'] \
                                        .astype(str) + \
                                            ' ' + \
                                                grouped_wbk['Shift End'] \
                                                    .astype(str)).dt \
                                                        .strftime('%Y-%m-%dT%H:%M:%S')

# convert to a list of dictionaries
# we can include 'Count' in the title so users see how many volunteers are there
calendar_data = []
for _, row in grouped_wbk.iterrows():
    calendar_data.append({
        "title": f"{row['Session description']}\n({row['Count']})", # use \n for the break
        "start": row['iso_start'],
        "end": row['iso_end'],
        "backgroundColor": site_colors.get(row['Session description'], "#6c757d"),
        "borderColor": "transparent",
        "allDay": False,
        "extendedProps": {
            "site": row['Session description'],
            "subregion": row['Subregion'],
            "volCount": row['Count'] # store the raw number just in case
        }
    })

# export to a JSON file for your website
with open('schedule.json', 'w') as f:
    json.dump(calendar_data, f)